In [1]:
import readdata
from text_vectorizer import CV
from text_vectorizer import TFIDF
from text_vectorizer import word2vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from text_vectorizer import outlierDection
import sys
import numpy as np

# Usage:
def getRemovedVals(X,Y = None,Ftype = "",isTest = False):

    X = np.array(X)
    index,_ = outlierDection(X,Ftype)
    if not isTest:
        Y = np.array(Y)
        Xrem,Yrem = removeOutliers(index,X,Y,Ftype)
        return Xrem,Yrem

    else:
        Xrem = removeOutliers(index,X,Y,Ftype)
        return Xrem



In [2]:
    arg = "cv"
    dfTrain = readdata.read_clean_data(readdata.TRAINFILEPATH, nolabel=False)
    dfTest = readdata.read_clean_data(readdata.TESTFILEPATH, nolabel=True)


In [3]:
    X_train = dfTrain['text'].to_numpy()
    Y_train = dfTrain['label'].to_numpy()
    X_test = dfTest['text'].to_numpy()
    
    X_train = X_train[:5000]
    Y_train = Y_train[:5000]
    X_test = X_test[:5000]
         

    if arg == "cv":
        X_train, X_test, _ = CV(X_train, X_test)  # train shape: (17973, 141221)
#         X_train,Y_train = getRemovedVals(X = X_train,Y = Y_train,Ftype = "CV_Train",isTest = False)
#         X_test = getRemovedVals(X = X_test,Y = None,Ftype = "CV_Test",isTest = True)


    elif arg == 'tfidf':
        X_train, X_test, _ = TFIDF(X_train, X_test)  # shape: (17973, 141221)
#         X_train,Y_train = getRemovedVals(X = X_train,Y = Y_train,Ftype = "TFIDF_Train",isTest = False)
#         X_test = getRemovedVals(X = X_test,Y = None,Ftype = "TFIDF_Test",isTest = True)
        
    elif arg == 'word2vec':
        X_train, X_test = word2vec(X_train, X_test)
#         X_train,Y_train = getRemovedVals(X = X_train,Y = Y_train,Ftype = "W2V_Train",isTest = False)
#         X_test = getRemovedVals(X = X_test,Y = None,Ftype = "W2V_Test",isTest = True)
        
    else:
        print("Error")




In [4]:
    # reshape input to be [samples, features]
    num_samples = X_train.shape[0]
    num_features = X_train.shape[1]
    X_train = np.reshape(np.array(X_train), (num_samples, num_features))

    # creating space for constant C
    c = np.logspace(0, 4, 10)
    # various solvers - only used solvers that supported L2
    solver = ['newton-cg', 'sag', 'lbfgs', 'liblinear']
    param_grid = dict(C=c, solver=solver)
    logistic = LogisticRegression(max_iter=500)


In [ ]:
    grid = GridSearchCV(logistic, param_grid=param_grid, cv=3, verbose=0)
    grid_result = grid.fit(X_train, Y_train)
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

c:\program files\python36\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [ ]:

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])